In [1]:
import sys
from pathlib import Path
import os

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")
    
# Set the environment variables from the file <root_dir>/.env
from mlfs import config
if os.path.exists(f"{root_dir}/.env"):
    settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /home/federica_lorenzini/ml-lab1/mlfs-book
HopsworksSettings initialized!


__This notebook should be scheduled to run daily__

The GitHub Action is stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

### Imports

In [2]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> Get the Sensor URL, Country, City, Street names from Hopsworks </span>

In [3]:
project = hopsworks.login()
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
location_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
location = json.loads(location_str)

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location_str

2025-11-16 15:23:32,455 INFO: Initializing external client
2025-11-16 15:23:32,457 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-16 15:23:34,831 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279175


'{"country": "Sweden", "city": "Sollentuna", "street": "ekmans-vag-11", "aqicn_url": "https://api.waqi.info/feed/@13983", "latitude": 59.43, "longitude": 17.95}'

### <span style="color:#ff5f27;">Get references to the Feature Groups </span>

In [4]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name='air_quality',
    version=3,
)
weather_fg = fs.get_feature_group(
    name='weather',
    version=1,
)

## <span style='color:#ff5f27'> Retrieve yesterday's air quality data (PM2.5) from the AQI API</span>


In [36]:
import requests
import pandas as pd

yesterday = today - datetime.timedelta(days=1)

# Air quality: yesterday
aq_yesterday_df = util.get_pm25(aqicn_url, country, city, street, yesterday, AQICN_API_KEY)

# lagged Air Quality for yesterday
lagged_days = 3
for i in range(1,lagged_days+1):
    lagged_days = yesterday - datetime.timedelta(days=i)
    aq_yesterday_df_lagged = util.get_pm25(aqicn_url, country, city, street, lagged_days, AQICN_API_KEY)
    idx='pm25_lagged_' + str(i)
    aq_yesterday_df[idx] = aq_yesterday_df_lagged['pm25']

print("Air Quality Data (yesterday):")
aq_yesterday_df

Air Quality Data (yesterday):


,pm25,country,city,street,date,url,pm25_lagged_1,pm25_lagged_2,pm25_lagged_3
0,10.0,Sweden,Sollentuna,ekmans-vag-11,2025-11-14,https://api.waqi.info/feed/@13983,10.0,10.0,10.0


In [37]:
aq_yesterday_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   pm25           1 non-null      float32       
 1   country        1 non-null      object        
 2   city           1 non-null      object        
 3   street         1 non-null      object        
 4   date           1 non-null      datetime64[ns]
 5   url            1 non-null      object        
 6   pm25_lagged_1  1 non-null      float32       
 7   pm25_lagged_2  1 non-null      float32       
 8   pm25_lagged_3  1 non-null      float32       
dtypes: datetime64[ns](1), float32(4), object(4)
memory usage: 184.0+ bytes


## <span style='color:#ff5f27'> Get Weather Forecast data</span>

In [38]:
# Download yesterday weather data
start_date = yesterday
end_date = yesterday

weather_yesterday_df = util.get_historical_weather(city=city, start_date=start_date, end_date=end_date, latitude=latitude, longitude=longitude)
weather_yesterday_df

Coordinates 59.43760681152344°N 18.04100227355957°E
Elevation 26.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-14,0.701583,0.0,16.293728,293.236481,Sollentuna


In [39]:
# Weather hourly forecast starting from today
weather_hourly_df = util.get_hourly_weather_forecast(city=city, latitude=latitude, longitude=longitude)
weather_hourly_df = weather_hourly_df.set_index('date')

# One daily prediction at 12:00
weather_daily_df = weather_hourly_df.between_time('11:59', '12:01')
weather_daily_df = weather_daily_df.reset_index()
weather_daily_df['date'] = pd.to_datetime(weather_daily_df['date']).dt.date
weather_daily_df['date'] = pd.to_datetime(weather_daily_df['date'])
weather_daily_df['city'] = city
weather_daily_df

Coordinates 59.5°N 18.0°E
Elevation 26.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-15,1.00,0.0,13.138765,260.537750,Sollentuna
1,2025-11-16,2.50,0.0,10.829958,248.552200,Sollentuna
2,2025-11-17,0.25,0.0,14.512064,316.004974,Sollentuna
3,2025-11-18,-0.90,0.0,7.208994,272.862335,Sollentuna
4,2025-11-19,0.40,0.0,7.952660,5.194350,Sollentuna
5,2025-11-20,1.15,1.7,11.019764,51.632477,Sollentuna
6,2025-11-21,1.25,0.0,13.324863,1.548122,Sollentuna


## <span style="color:#ff5f27;">Uploading new data to the Feature Store</span>

In [40]:
# Insert new data
air_quality_fg.insert(aq_yesterday_df)
weather_fg.insert(weather_yesterday_df, wait=True)
weather_fg.insert(weather_daily_df, wait=True)
print("Updated feature groups with yesterday's data and forecast.")


2025-11-15 16:01:28,267 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279175/fs/1265791/fg/1717615


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_3_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279175/jobs/named/air_quality_3_offline_fg_materialization/executions
2025-11-15 16:01:42,331 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279175/fs/1265791/fg/1703315


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:00 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279175/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-15 16:01:58,014 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-15 16:02:01,246 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-15 16:03:28,090 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-15 16:03:28,243 INFO: Waiting for log aggregation to finish.
2025-11-15 16:03:40,265 INFO: Execution finished successfully.
2025-11-15 16:03:40,445 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1279175/fs/1265791/fg/1703315


Uploading Dataframe: 100.00% |██████████| Rows 7/7 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1279175/jobs/named/weather_1_offline_fg_materialization/executions
2025-11-15 16:03:56,634 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-15 16:03:59,839 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-15 16:05:36,670 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-15 16:05:36,846 INFO: Waiting for log aggregation to finish.
2025-11-15 16:05:45,671 INFO: Execution finished successfully.
Updated feature groups with yesterday's data and forecast.
